In [4]:
"""Forecasting the total sales (Value) for the next 3 months"""
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pandas as pd

def forecast_sales(df):
    # Group data by Month-Year and sum Total_Value
    time_series = df.groupby('Month-Year')['Total_Value'].sum().reset_index()

    # Parse Month-Year with textual month format
    time_series['Month-Year'] = pd.to_datetime(time_series['Month-Year'], format='%B %Y', errors='coerce')

    # Drop rows where date parsing failed
    if time_series['Month-Year'].isna().any():
        print("Warning: Some dates could not be parsed and were dropped.")
        time_series = time_series.dropna(subset=['Month-Year'])

    # Set index to Month-Year and sort
    time_series = time_series.set_index('Month-Year').sort_index()['Total_Value']

    # Set frequency to monthly
    time_series = time_series.asfreq('MS')

    # Handle missing values in the time series
    if time_series.isna().any():
        print("Warning: Missing data detected. Filling missing values with linear interpolation.")
        time_series = time_series.interpolate(method='linear')

    # Handle insufficient data points
    if len(time_series) < 24:
        print("Warning: Not enough data for seasonal decomposition. Using a non-seasonal model.")
        model = ExponentialSmoothing(time_series, trend='add', seasonal=None, initialization_method="estimated")
    else:
        model = ExponentialSmoothing(
            time_series,
            trend='add',
            seasonal='add',
            seasonal_periods=12,
            initialization_method="estimated"
        )

    try:
        # Fit the model
        fitted_model = model.fit(optimized=True)
    except Exception as e:
        print(f"Error fitting the model: {e}")
        return None

    # Forecast next 3 months
    forecast = fitted_model.forecast(steps=3)
    return forecast

# usage
if __name__ == "__main__":
    try:
        # Loading your dataset
        df = pd.read_csv("../cleaned_data.csv")

        # Forecasting sales
        sales_forecast = forecast_sales(df)

        if sales_forecast is not None:
            print("\n##### Next 3 months forecast:")
            print(sales_forecast)
        else:
            print("Forecasting failed.")
    except FileNotFoundError:
        print("Error: The file '../cleaned_data.csv' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")



##### Next 3 months forecast:
2025-01-01    1.640576e+08
2025-02-01    1.660495e+08
2025-03-01    1.680414e+08
Freq: MS, dtype: float64


/home/moses/.local/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
